# Tensorflow intro

This tutorial shows the basic usage of tensorflow to train neural networks

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Simple NN for classification

## Data processing
Read data and convert them to numerical inputs

In [3]:
X, y = make_classification(n_samples=20000, n_features=8, n_informative=5, 
                           n_redundant=0, n_classes=2, random_state=1)

In [4]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print('train size: {}, target_ratio: {:.3f}'.format(X_train.shape, np.mean(y_train)))
print('test size: {}, target_ratio: {:.3f}'.format(X_test.shape, np.mean(y_test)))
print('dev size: {}, target_ratio: {:.3f}'.format(X_dev.shape, np.mean(y_dev)))

train size: (12800, 8), target_ratio: 0.502
test size: (4000, 8), target_ratio: 0.501
dev size: (3200, 8), target_ratio: 0.502


## Building a simple model with tf.keras

Very useful documentations with many examples and detailed explanation of everything you might need:
 - https://www.tensorflow.org/api_docs/python/tf/keras/
 - https://keras.io/api/

Contain everything about:
  - Model building: Activations, Losses, Optimizers, Regularization
  - Data processing
  - Pretrained models and datasets
  - Automatic differentiation
  - ...

  

### Model speficication

three APIs for building the model
   - sequential - easy to code, but less flexible - we will use it sometimes
   - functional - flexible and still easy to code - we will use it the most
   - model subclassing - rather complicated and not very much used - we will skip it

#### Sequential API

(https://www.tensorflow.org/guide/keras/sequential_model)

Easy to code but not appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [7]:
# Specification A)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([X_train.shape[1],]), # Create input layer with 'input data' neurons
    tf.keras.layers.Dense(10, activation="relu"), # Create hidden layer with 10 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation="sigmoid"), # Create output layer with one neuron and sigmoid activation
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                90        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Specification B)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                90        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


#### Functional API

(https://www.tensorflow.org/guide/keras/functional)

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.



In [9]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

hidden = tf.keras.layers.Dense(10)(inputs)
hidden = tf.keras.activations.relu(hidden)
hidden = tf.keras.layers.Dense(1)(hidden)
outputs = tf.keras.activations.sigmoid(hidden)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Model')

In [10]:
model.summary()

Model: "Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                90        
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 10)]              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
_________________________________________________________________
tf_op_layer_Sigmoid (TensorF [(None, 1)]               0         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


### Model compilation and training

In [11]:
# compile the model with selected optimizer, loss and metrics
model.compile(
        optimizer=tf.optimizers.Adam(), # Several other possibilities for optimizers 
        loss=tf.losses.BinaryCrossentropy(), # Select the proper loss for the task
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()], # Select the proper metrics for the task
)

In [12]:
print('\n>>> Bias of the last layers:')
print(model.layers[3].weights[1].numpy())

print('\n>>> Kernel of the last layers:')
print(model.layers[3].weights[0].numpy())

print('\n>>> Bias of the first layers:')
print(model.layers[1].weights[1].numpy())

print('\n>>> Kernel of the first layers:')
print(model.layers[1].weights[0].numpy())


>>> Bias of the last layers:
[0.]

>>> Kernel of the last layers:
[[ 0.04851639]
 [-0.24508768]
 [-0.13187832]
 [ 0.05884331]
 [-0.15282243]
 [-0.3866087 ]
 [ 0.6848112 ]
 [-0.22949755]
 [-0.09657723]
 [-0.64680386]]

>>> Bias of the first layers:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

>>> Kernel of the first layers:
[[-0.49263224 -0.32091686  0.23436737 -0.01964736  0.38018882  0.33966905
  -0.12686235  0.47103775 -0.24198651 -0.12971807]
 [ 0.15010089 -0.27177432  0.37964827 -0.37813812  0.45467758  0.00761402
  -0.17108136 -0.09188083  0.00311667  0.01995319]
 [ 0.21115613  0.556085    0.523522   -0.42312437  0.29241908  0.17406416
   0.01183522 -0.32265553  0.22161222 -0.02221262]
 [ 0.3853857   0.13021207  0.11804235 -0.05382901  0.42697048 -0.35721403
  -0.07440573  0.1814974   0.37874871 -0.18781891]
 [ 0.40240794  0.05259258  0.22057366 -0.03655571 -0.22361258  0.27079827
  -0.35875314  0.49570286  0.02230388  0.5181092 ]
 [ 0.43708265 -0.05469453 -0.14172357 -0.35890868  0.17780411

In [13]:
# train the model with default setting
model.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100
200/200 [==============================] - 0s 756us/step - loss: 0.5614 - auc: 0.8051 - binary_accuracy: 0.7152
Epoch 2/100
200/200 [==============================] - 0s 642us/step - loss: 0.4301 - auc: 0.8970 - binary_accuracy: 0.8218
Epoch 3/100
200/200 [==============================] - 0s 708us/step - loss: 0.3773 - auc: 0.9196 - binary_accuracy: 0.8535
Epoch 4/100
200/200 [==============================] - 0s 643us/step - loss: 0.3431 - auc: 0.9329 - binary_accuracy: 0.8712
Epoch 5/100
200/200 [==============================] - 0s 643us/step - loss: 0.3200 - auc: 0.9405 - binary_accuracy: 0.8810
Epoch 6/100
200/200 [==============================] - 0s 788us/step - loss: 0.3044 - auc: 0.9461 - binary_accuracy: 0.8866
Epoch 7/100
200/200 [==============================] - 0s 808us/step - loss: 0.2942 - auc: 0.9487 - binary_accuracy: 0.8898
Epoch 8/100
200/200 [==============================] - 0s 643us/step - loss: 0.2877 - auc: 0.9505 - binary_accuracy: 0.8931
Epoch 9/

In [14]:
# Evaluate the model and predict for the test data
model.evaluate(X_test, y_test)
test_pred = model.predict(X_test)

125/125 [==============================] - 0s 654us/step - loss: 0.2349 - auc: 0.9681 - binary_accuracy: 0.9120


In [15]:
for pred, true in zip(test_pred, y_test[0:10]):
    print('{} - {:.2f}'.format(true, pred[0]))

1 - 0.49
0 - 0.34
0 - 0.26
1 - 1.00
0 - 0.00
0 - 0.00
1 - 0.97
0 - 0.06
1 - 0.93
1 - 1.00


### Add early stopping and regularization

In [16]:
# Input layer
inputs = tf.keras.Input(shape=(X_train.shape[1]))

# Hidden layer with regularization and ReLU
hidden = tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs)
hidden = tf.keras.activations.relu(hidden)

# Output layer with regularization and sigmoid
outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.001))(hidden)
outputs = tf.keras.activations.sigmoid(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

model.compile(
        optimizer=tf.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()],
)

model.summary()

Model: "RegularizedModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                90        
_________________________________________________________________
tf_op_layer_Relu_1 (TensorFl [(None, 10)]              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
_________________________________________________________________
tf_op_layer_Sigmoid_1 (Tenso [(None, 1)]               0         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [17]:
epochs = 200

early_call = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_dev, y_dev),
          callbacks=[early_call])

Epoch 1/200
400/400 [==============================] - 1s 2ms/step - loss: 0.5726 - auc_1: 0.7799 - binary_accuracy: 0.6976 - val_loss: 0.4372 - val_auc_1: 0.9073 - val_binary_accuracy: 0.8169
Epoch 2/200
400/400 [==============================] - 0s 850us/step - loss: 0.3920 - auc_1: 0.9237 - binary_accuracy: 0.8449 - val_loss: 0.3556 - val_auc_1: 0.9353 - val_binary_accuracy: 0.8628
Epoch 3/200
400/400 [==============================] - 0s 757us/step - loss: 0.3404 - auc_1: 0.9411 - binary_accuracy: 0.8752 - val_loss: 0.3253 - val_auc_1: 0.9448 - val_binary_accuracy: 0.8803
Epoch 4/200
400/400 [==============================] - 0s 758us/step - loss: 0.3167 - auc_1: 0.9484 - binary_accuracy: 0.8884 - val_loss: 0.3086 - val_auc_1: 0.9504 - val_binary_accuracy: 0.8872
Epoch 5/200
400/400 [==============================] - 0s 755us/step - loss: 0.3019 - auc_1: 0.9531 - binary_accuracy: 0.8960 - val_loss: 0.2962 - val_auc_1: 0.9545 - val_binary_accuracy: 0.8969
Epoch 6/200
400/400 [======

400/400 [==============================] - 0s 723us/step - loss: 0.2365 - auc_1: 0.9766 - binary_accuracy: 0.9270 - val_loss: 0.2411 - val_auc_1: 0.9762 - val_binary_accuracy: 0.9237
Epoch 83/200
400/400 [==============================] - 0s 723us/step - loss: 0.2364 - auc_1: 0.9768 - binary_accuracy: 0.9267 - val_loss: 0.2407 - val_auc_1: 0.9763 - val_binary_accuracy: 0.9216
Epoch 84/200
400/400 [==============================] - 0s 705us/step - loss: 0.2361 - auc_1: 0.9769 - binary_accuracy: 0.9268 - val_loss: 0.2407 - val_auc_1: 0.9762 - val_binary_accuracy: 0.9216
Epoch 85/200
400/400 [==============================] - 0s 723us/step - loss: 0.2359 - auc_1: 0.9769 - binary_accuracy: 0.9273 - val_loss: 0.2414 - val_auc_1: 0.9761 - val_binary_accuracy: 0.9228
Epoch 86/200
400/400 [==============================] - 0s 704us/step - loss: 0.2360 - auc_1: 0.9770 - binary_accuracy: 0.9283 - val_loss: 0.2409 - val_auc_1: 0.9763 - val_binary_accuracy: 0.9234
Epoch 87/200
400/400 [===========

### Gridsearch and Tensorboard
Run gridsearch over hidden layer size, L2 regularization, activation, check the outputs in Tensorboard

I recommend not to run Tensorboard from Jupyter notebook but from terminal directly

use "tensorboard --logdir logs" in command line

In [ ]:
# hidden_sizes = [2, 5, 10, 20, 50]
# l2_regs = [0.01, 0.001, 0.0001]
# activations = ['relu', 'tanh']

hidden_sizes = [2, 5]
l2_regs = [0.01]
activations = ['relu', 'tanh']

epochs = 200
batch_size = 64

early_call = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', patience=10, restore_best_weights=True)

for activation in activations:
    for l2_reg in l2_regs:
        for hidden_size in hidden_sizes:
            if activation == 'relu':
                activate = tf.keras.activations.relu
            elif activation == 'tanh':
                activate = tf.keras.activations.tanh

            # Create Tensorboard Callback
            param_string = 'act-{},l2-{},hs-{}'.format(activation, l2_reg, hidden_size)
            log_dir = 'logs/binary_classification_test/' + param_string
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            # Input layer
            inputs = tf.keras.Input(shape=(X_train.shape[1]))

            # Hidden layer with regularization and ReLU
            hidden = tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
            hidden = activate(hidden)

            # Output layer with regularization and sigmoid
            outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(hidden)
            outputs = tf.keras.activations.sigmoid(outputs)

            model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

            model.compile(
                    optimizer=tf.optimizers.Adam(),
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy()],
            )

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, y_dev),
                      callbacks=[early_call, tensorboard_callback])

Epoch 1/200
  1/200 [..............................] - ETA: 0s - loss: 1.5168 - AUC: 0.2839 - binary_accuracy: 0.4062WARNING:tensorflow:From C:\Users\karel\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
200/200 [==============================] - 1s 3ms/step - loss: 1.1487 - AUC: 0.3033 - binary_accuracy: 0.3989 - val_loss: 0.9585 - val_AUC: 0.3134 - val_binary_accuracy: 0.3825
Epoch 2/200
200/200 [==============================] - 0s 1ms/step - loss: 0.8560 - AUC: 0.3694 - binary_accuracy: 0.4019 - val_loss: 0.7970 - val_AUC: 0.3838 - val_binary_accuracy: 0.4241
Epoch 3/200
200/200 [==============================] - 0s 823us/step - loss: 0.7555 - AUC: 0.4408 - binary_accuracy: 0.4847 - val_loss: 0.7331 - val_AUC: 0.4696 - val_binary_accuracy: 0.5181
Epoch 4/200
200/200 [=======================

Epoch 83/200
200/200 [==============================] - 0s 878us/step - loss: 0.4771 - AUC: 0.8896 - binary_accuracy: 0.8181 - val_loss: 0.4676 - val_AUC: 0.8952 - val_binary_accuracy: 0.8272
Epoch 84/200
200/200 [==============================] - 0s 873us/step - loss: 0.4770 - AUC: 0.8895 - binary_accuracy: 0.8187 - val_loss: 0.4676 - val_AUC: 0.8956 - val_binary_accuracy: 0.8256
Epoch 85/200
200/200 [==============================] - 0s 838us/step - loss: 0.4770 - AUC: 0.8895 - binary_accuracy: 0.8182 - val_loss: 0.4679 - val_AUC: 0.8953 - val_binary_accuracy: 0.8266
Epoch 86/200
200/200 [==============================] - 0s 1ms/step - loss: 0.4769 - AUC: 0.8896 - binary_accuracy: 0.8179 - val_loss: 0.4676 - val_AUC: 0.8956 - val_binary_accuracy: 0.8272
Epoch 87/200
200/200 [==============================] - 0s 1ms/step - loss: 0.4769 - AUC: 0.8896 - binary_accuracy: 0.8194 - val_loss: 0.4675 - val_AUC: 0.8955 - val_binary_accuracy: 0.8281
Epoch 88/200
200/200 [======================

Epoch 168/200
200/200 [==============================] - 0s 878us/step - loss: 0.4754 - AUC: 0.8910 - binary_accuracy: 0.8195 - val_loss: 0.4660 - val_AUC: 0.8973 - val_binary_accuracy: 0.8278
Epoch 169/200
200/200 [==============================] - 0s 871us/step - loss: 0.4754 - AUC: 0.8910 - binary_accuracy: 0.8191 - val_loss: 0.4659 - val_AUC: 0.8971 - val_binary_accuracy: 0.8272
Epoch 170/200
200/200 [==============================] - 0s 871us/step - loss: 0.4753 - AUC: 0.8911 - binary_accuracy: 0.8187 - val_loss: 0.4660 - val_AUC: 0.8970 - val_binary_accuracy: 0.8256
Epoch 171/200
200/200 [==============================] - 0s 887us/step - loss: 0.4753 - AUC: 0.8911 - binary_accuracy: 0.8202 - val_loss: 0.4660 - val_AUC: 0.8971 - val_binary_accuracy: 0.8225
Epoch 172/200
200/200 [==============================] - 0s 923us/step - loss: 0.4752 - AUC: 0.8912 - binary_accuracy: 0.8180 - val_loss: 0.4662 - val_AUC: 0.8970 - val_binary_accuracy: 0.8281
Epoch 173/200
200/200 [============

200/200 [==============================] - 0s 968us/step - loss: 0.6264 - AUC: 0.7798 - binary_accuracy: 0.7192 - val_loss: 0.5984 - val_AUC: 0.8074 - val_binary_accuracy: 0.7516
Epoch 6/200
200/200 [==============================] - 0s 1ms/step - loss: 0.5825 - AUC: 0.8188 - binary_accuracy: 0.7548 - val_loss: 0.5604 - val_AUC: 0.8359 - val_binary_accuracy: 0.7769
Epoch 7/200
200/200 [==============================] - 0s 1ms/step - loss: 0.5520 - AUC: 0.8403 - binary_accuracy: 0.7770 - val_loss: 0.5341 - val_AUC: 0.8512 - val_binary_accuracy: 0.7969
Epoch 8/200
200/200 [==============================] - 0s 2ms/step - loss: 0.5320 - AUC: 0.8510 - binary_accuracy: 0.7877 - val_loss: 0.5167 - val_AUC: 0.8594 - val_binary_accuracy: 0.8053
Epoch 9/200
200/200 [==============================] - 0s 1ms/step - loss: 0.5194 - AUC: 0.8572 - binary_accuracy: 0.7917 - val_loss: 0.5058 - val_AUC: 0.8649 - val_binary_accuracy: 0.8097
Epoch 10/200
200/200 [==============================] - 0s 1ms/st

Epoch 51/200
200/200 [==============================] - 0s 825us/step - loss: 0.4504 - AUC: 0.9366 - binary_accuracy: 0.8690 - val_loss: 0.4456 - val_AUC: 0.9405 - val_binary_accuracy: 0.8797
Epoch 52/200
200/200 [==============================] - 0s 888us/step - loss: 0.4502 - AUC: 0.9365 - binary_accuracy: 0.8683 - val_loss: 0.4455 - val_AUC: 0.9404 - val_binary_accuracy: 0.8794
Epoch 53/200
200/200 [==============================] - 0s 1ms/step - loss: 0.4502 - AUC: 0.9370 - binary_accuracy: 0.8684 - val_loss: 0.4454 - val_AUC: 0.9400 - val_binary_accuracy: 0.8781
Epoch 54/200
200/200 [==============================] - 0s 933us/step - loss: 0.4503 - AUC: 0.9368 - binary_accuracy: 0.8687 - val_loss: 0.4456 - val_AUC: 0.9404 - val_binary_accuracy: 0.8794
Epoch 55/200
200/200 [==============================] - 0s 918us/step - loss: 0.4501 - AUC: 0.9366 - binary_accuracy: 0.8690 - val_loss: 0.4456 - val_AUC: 0.9410 - val_binary_accuracy: 0.8806
Epoch 56/200
 82/200 [===========>........